#### 1. Data Loading and Preprocessing

In [1]:
from config import *
from LoadDataset import CustomImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from config import dataset_test_path, dataset_train_path, dataset_val_path

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CustomImageDataset(file_path=dataset_train_path, folder_path=dataset_root, transform=transform)
val_dataset = CustomImageDataset(file_path=dataset_val_path, folder_path=dataset_root, transform=transform)
test_dataset = CustomImageDataset(file_path=dataset_test_path, folder_path=dataset_root, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Festplatte/MATIML/data/clothing1M/annotations/category_names_eng.txt'

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset), "total:", len(train_dataset) + len(val_dataset) + len(test_dataset)

(674373, 207499, 155625, 'total:', 1037497)

##### 2. Model Definition

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNet50(nn.Module):
    def __init__(self, num_classes=14):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        # TODO: research if there are other weights to be set (like V2 if it exists)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50(num_classes=len(class_names))
model = model.to(device)


In [ ]:
# import torch
# from torch.optim import lr_scheduler
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import models
# import time
# import copy

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# weights = models.ResNet50_Weights.IMAGENET1K_V1
# model = models.resnet50(weights=weights)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(class_names))
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


#### 3. Prototype Selection

In [ ]:
import numpy as np

def select_prototypes(features, labels, num_prototypes=8):
    prototypes = {}
    for label in np.unique(labels):
        class_features = features[labels == label]
        similarity_matrix = np.matmul(class_features, class_features.T)
        densities = np.sum(similarity_matrix > 0.5, axis=1)
        sorted_indices = np.argsort(-densities)[:num_prototypes]
        prototypes[label] = class_features[sorted_indices]
    return prototypes
# TODO: let chatgpt explain this code

##### 4. Label Correction

In [ ]:
def correct_labels(features, prototypes):
    corrected_labels = []
    for feature in features:
        max_similarity = -1
        corrected_label = -1
        for label, proto_features in prototypes.items():
            similarities = np.mean([np.dot(feature, proto_feature) for proto_feature in proto_features])
            if similarities > max_similarity:
                max_similarity = similarities
                corrected_label = label
        corrected_labels.append(corrected_label)
    return np.array(corrected_labels)
# TODO: let chatgpt explain the code + randomly_samples_img_size (1280) randomly sampled images and the current image need to be feature extracted for the clustering & prototype selection and the label correction


#### 5. Training Loop with Iterative Self-Learning

In [ ]:
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

def train_model(model, train_loader, num_epochs=num_epochs, learning_rate=lr, momentum=momentum, weight_decay=weight_decay, step_size=step_size, gamma=gamma, alpha=alpha):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        writer.add_scalar('Loss/train', running_loss/len(train_loader), epoch)
        
        if epoch % 5 == 0:
            model.eval()
            all_features = []
            all_labels = []
            with torch.no_grad():
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    features = model(inputs)
                    all_features.append(features.cpu().numpy())
                    all_labels.append(labels.cpu().numpy())
            
            all_features = np.concatenate(all_features)
            all_labels = np.concatenate(all_labels)
            
            prototypes = select_prototypes(all_features, all_labels, num_prototypes=num_prototypes)
            corrected_labels = correct_labels(all_features, prototypes)
            
            for inputs, labels, corrected in zip(train_loader, all_labels, corrected_labels):
                inputs, labels, corrected = inputs.to(device), labels.to(device), corrected.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = (1 - alpha) * criterion(outputs, labels) + alpha * criterion(outputs, corrected)
                loss.backward()
                optimizer.step()
        
        scheduler.step()
    
    writer.close()

In [19]:
train_model(model, train_loader)

ValueError: too many values to unpack (expected 2)